In [ ]:
from types import SimpleNamespace
import kwant
import numpy as np
import holoviews as hv
from supercurrent import current_at_phase, null_H
hv.notebook_extension()

s0 = np.array([[1., 0.], [0., 1.]])
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1j], [1j, 0.]])
sz = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz
onsite_lead = lambda site, p: -p.mu * sz + p.delta * sx
hops = lambda site1, site2, p: -p.t * sz

def make_test_system(W, L):
    lat = kwant.lattice.square()
    syst = kwant.Builder()
    range_w = range(W//2 - W + 1, W//2 + 1)
    range_l = range(L//2 - L + 1, L//2 + 1)
    syst[(lat(x, y) for x in range_l for y in range_w)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(0, y) for y in range_w]
    r_cut = [lat(1, y) for y in range_w]

    num_orbs = 2
    dim = num_orbs * len(l_cut) * len(r_cut)
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0)))
    lead[(lat(0, y) for y in range_w)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

In [ ]:
from supercurrent import current_at_phase, null_H
W = 2
L = 2
delta = 0.01
T = 0.5 * delta
matsfreqs = 50

syst, hopping = make_test_system(W, L)
p = SimpleNamespace(mu=0., t=1., delta=delta)

phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=0.01) for phase in phases])
plt1 = hv.Curve(I/delta/W, kdims=['phase'], vdims=['$I$'])
plt1